<a href="https://colab.research.google.com/github/UniVR-DH/ADHLab/blob/main/lecture06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testing Named Entity Recognition

We will use the Spacy Library:
https://spacy.io/usage/spacy-101


<img src="https://drive.google.com/uc?export=view&id=1m_EMdnI5C826kgqK7r5vB4TXnB0-Wq7W" alt="Intestazione con loghi istituzionali" width="525"/>

| Docente      | Insegnamento | Anno Accademico    |
| :---        |    :----   |          ---: |
| Matteo Lissandrini      | Laboratorio Avanzato di Informatica Umanistica       | 2023/2024   |

## Usual install and basic imports

In [1]:
%pip install wikipedia-api
%pip install spacy==3.7.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.6.1
    Uninstalling spacy-3.6.1:
      Successfully uninstalled spacy-3.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.6.0 requires spacy<3.7.0,>=3.6.0, but you have spacy 3.7.0 which is incompatible.


In [5]:
import wikipediaapi
import re

# Step 1: Import necessary libraries
#wiki_wiki = wikipediaapi.Wikipedia('en', user_agent='YourUserAgent/1.0')
wiki_wiki = wikipediaapi.Wikipedia('en', extract_format=wikipediaapi.ExtractFormat.WIKI, user_agent='YourUserAgent/1.0')

# Step 2: Prepare a list of titles of Wikipedia pages
fantasy_literature_titles = [
    "Fantasy literature",
    "The Lord of the Rings",
    "Harry Potter",
    "A Song of Ice and Fire",
    "The Chronicles of Narnia",
    "The Hobbit",
    "Alice's Adventures in Wonderland",
    "The Wizard of Oz",
    "The Silmarillion",
    "Discworld",
    "His Dark Materials",
    "The Wheel of Time",
    "Earthsea",
    "The Once and Future King",
    "The Princess Bride",
    "The Name of the Wind",
    "Mistborn",
    "The Malazan Book of the Fallen",
    "The Kingkiller Chronicle",
    "The Inheritance Cycle"
]

# Step 3: Write code to download main content in plain text and create a collection of cleaned text
cleaned_texts = []

for title in fantasy_literature_titles:
    page_py = wiki_wiki.page(title)

    if page_py.exists():
        # Keep the original URL of the page
        original_url = page_py.fullurl

        # Remove brackets and their contents from the text
        content = re.sub(r'\[[^\]]*\]', '', page_py.text)

        # Remove newline characters and extra spaces
        clean_text = ' '.join(content.split())

        # Store the cleaned text along with the original URL
        cleaned_texts.append({'title': title, 'content': clean_text, 'url': original_url})
    else:
        print(f"Page '{title}' does not exist on Wikipedia.")

# Display the cleaned texts and their original URLs (optional)
for idx, entry in enumerate(cleaned_texts, 1):
    print(f"Text {idx} - Title: {entry['title']}\nURL: {entry['url']}\nContent:\n{entry['content']}\n{'='*50}\n")


TypeError: ignored